In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from utils import load_wav_to_torch, load_filepaths_and_text
from mel_processing import spectrogram_torch,spec_to_mel_torch

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
def get_mel(filename):
    audio, sampling_rate = load_wav_to_torch(filename)
    print(sampling_rate)
    audio_norm = audio / 32768.0
    audio_norm = audio_norm.unsqueeze(0)
    spec = spectrogram_torch(audio_norm, 1024, 24000, 320, 1024,center=False)
    spec = torch.squeeze(spec, 0)
    ### 提取梅尔特征
    mel = spec_to_mel_torch(spec, 1024, 80, 24000, 0.0, None)
    return mel
def load_checkpoint(checkpoint_path, model, optimizer=None):
  assert os.path.isfile(checkpoint_path)
  checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
  iteration = checkpoint_dict['iteration']
  learning_rate = checkpoint_dict['learning_rate']
  print('222222222222222222222222')
  if optimizer is not None:
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
  saved_state_dict = checkpoint_dict['model']
  if hasattr(model, 'module'):
    state_dict = model.module.state_dict()
  else:
    state_dict = model.state_dict()
  new_state_dict= {}
  for k, v in state_dict.items():
    try:
      new_state_dict[k] = saved_state_dict[k]
    except:
      print("%s is not in the checkpoint" % k)
      new_state_dict[k] = v
  if hasattr(model, 'module'):
    model.module.load_state_dict(new_state_dict)
  else:
    model.load_state_dict(new_state_dict)
  print("Loaded checkpoint '{}' (iteration {})" .format(
    checkpoint_path, iteration))
  return model, optimizer, learning_rate, iteration

## LJ Speech

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/path/to/pretrained_ljs.pth", net_g, None)

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

## VCTK

In [2]:
# hps = utils.get_hparams_from_file("./logs/promptstyle_s1/config.json")
hps = utils.get_hparams_from_file("/data1/pmy/jlpro/vits/configs/libritts.json")

In [4]:
style_wavfp = '/data1/pmy/jlpro/EI_VC/infer_acl/test_data/260_123288_000007_000000/260_123288_000007_000000.wav'
mel = get_mel(style_wavfp)
mel = mel.cuda(device='cuda:1').unsqueeze(0)
print(mel.shape)

24000
torch.Size([1, 80, 979])


In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda(device='cuda:0')
_ = net_g.eval()

# all_modules = {}
# all_params = {}
# for name, param in net_g.named_parameters():
#     module_name = name.split('.')[0]
#     if module_name not in all_modules:
#         all_modules[module_name] = []
#     all_modules[module_name].append(param)
# for k,v in all_modules.items():
#     cur_module_param = sum([np.prod(p.size()) for p in v]) / 1_000_000
#     all_params[k] = cur_module_param

# for k,v in all_params.items():
#     print(f'| {k} Trainable Parameters: %.3fM' % v)

# for name, param in net_g.named_parameters():
#     print(f"Parameter name: {name}, Size: {param.size()}")
# state_dict = net_g.state_dict()
# for k,v in state_dict.items():
#     if k.startswith('style_adapter'):
#         print(k)
# print(state_dict.keys())

_ = utils.load_checkpoint("/data1/pmy/jlpro/vits/logs/prompt_style_baseline/G_1365000.pth", net_g, None)

/home/pmy/anaconda3/envs/pro/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


use_sdp **************************************************
use_style_encoder **************************************************


In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("/data1/pmy/jlpro/EI_VC/checkpoints/bert-base-uncased")
model = AutoModel.from_pretrained("/data1/pmy/jlpro/EI_VC/checkpoints/bert-base-uncased")      
def get_style_embed(style_prompt):
    inputs = tokenizer(style_prompt, return_tensors="pt")
    outputs = model(**inputs)
    return outputs[-1]

/home/pmy/anaconda3/envs/pro/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pmy/anaconda3/envs/pro/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
stn_tst = get_text("But it is not with a view to distinction that you should cultivate this talent, if you consult your own happiness.", hps)
# style_wavfp = '/data1/pmy/jlpro/EI_VC/data/raw/LibriTTS/train-other-500/25/88353/25_88353_000001_000000.wav'
# mel = get_mel(style_wavfp)
mel = torch.load('/data1/pmy/jlpro/vits/DUMMY2/3728/105386/3728_105386_000005_000000.mel.pt')
print(mel.shape)
style_prompt = "A speaker with low energy, and with slow speed."
style_embed = get_style_embed(style_prompt)
print(style_embed.shape)

with torch.no_grad():
    x_tst = stn_tst.cuda(device='cuda:0').unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda(device='cuda:0')
    sid = torch.LongTensor([7]).cuda(device='cuda:0')
    style_embed = style_embed.cuda(0)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid,mel=mel,style_embed=style_embed, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    print(audio.shape)
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

torch.Size([80, 651])
torch.Size([1, 768])
(146241,)


### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))